<a href="https://colab.research.google.com/github/codingniket/Python-Training/blob/main/19_12_2025/Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

In [2]:
data = [
    ("UI",50),
    ("U2",80),
]

df = spark.createDataFrame(data,["name","score"])
df.show()

+----+-----+
|name|score|
+----+-----+
|  UI|   50|
|  U2|   80|
+----+-----+



UDF = User Define Function


1.   Only use when really needed spark nativly works on Java
2.  It doesnt understand udf so it is slowly and impact performance

3. Use when and otherwise when need to use function


In [3]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def classify(score):
  if score >= 80:
    return "High"
  elif score >= 60:
    return "Medium"
  else:
    return "Low"

classify_udf = udf(classify, StringType())

df.withColumn("grade", classify_udf(col("score"))).show()

+----+-----+-----+
|name|score|grade|
+----+-----+-----+
|  UI|   50|  Low|
|  U2|   80| High|
+----+-----+-----+



In [8]:
from pyspark.sql.functions import when,col

df = df.withColumn("grade",
    when(col("score") >= 80, "High")
    .when(col("score") >= 60, "Medium")
    .otherwise("Low")
)

In [7]:
df.orderBy(df.score.desc()).show()

+----+-----+
|name|score|
+----+-----+
|  U2|   80|
|  UI|   50|
+----+-----+



In [6]:
df.withColumn("grade", classify_udf(col("score"))).explain(True)

== Parsed Logical Plan ==
'Project [unresolvedstarwithcolumns(grade, classify('score)#40, None)]
+- LogicalRDD [name#0, score#1L], false

== Analyzed Logical Plan ==
name: string, score: bigint, grade: string
Project [name#0, score#1L, classify(score#1L)#40 AS grade#41]
+- LogicalRDD [name#0, score#1L], false

== Optimized Logical Plan ==
Project [name#0, score#1L, pythonUDF0#42 AS grade#41]
+- BatchEvalPython [classify(score#1L)#40], [pythonUDF0#42]
   +- LogicalRDD [name#0, score#1L], false

== Physical Plan ==
*(2) Project [name#0, score#1L, pythonUDF0#42 AS grade#41]
+- BatchEvalPython [classify(score#1L)#40], [pythonUDF0#42]
   +- *(1) Scan ExistingRDD[name#0,score#1L]



In [9]:
df.orderBy("grade",df.score.desc()).show()

+----+-----+-----+
|name|score|grade|
+----+-----+-----+
|  U2|   80| High|
|  UI|   50|  Low|
+----+-----+-----+



Learning about Set Opearation

In [13]:
data_a = [
    ("U1","Java"),
    ("U2","Python"),
    ("U3","Scala"),

]

data_b = [
    ("U1","Java"),
    ("U4","Python"),
    ("U5","Scala"),

]

df_a = spark.createDataFrame(data_a,["user","course"])
df_b = spark.createDataFrame(data_b,["user","course"])



In [14]:
df_a.union(df_b).show()

+----+------+
|user|course|
+----+------+
|  U1|  Java|
|  U2|Python|
|  U3| Scala|
|  U1|  Java|
|  U4|Python|
|  U5| Scala|
+----+------+



In [15]:
df_a.intersect(df_b).show()

+----+------+
|user|course|
+----+------+
|  U1|  Java|
+----+------+

